In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

In [5]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informação geral e detalhada no google sobre um tópico.",
        tools=[google_search],
        instruction="""
       Função: Agente especializado em buscar e coletar dados e informações
       relevantes em (Google Search).
       Fornecer pesquisa detalhadas e relevantes para o planejamento e organização de conteúdo.
       Oferecer apenas as pesquisas mais relevantes.
       **Instruções de Busca Específicas:**
       1. **Foco Principal:** Busque por **informações aprofundadas, dados, fatos, tendências, contextos e detalhes** sobre o tópico fornecido. O objetivo é obter um entendimento completo e factual do assunto.
       2. **Conteúdo a Excluir Ativamente:**
          - Cursos online, treinamentos, webinars, plataformas de ensino à distância, materiais de curso.
       3. **Recomendações (Livros, etc.):** Se encontrar recomendações relevantes (como livros) que **complementam ou ilustram a informação geral** sobre o tópico, pode incluí-las. No entanto, o foco **não** é criar uma lista de recomendações, mas sim usar essas recomendações como exemplos ou fontes dentro de um contexto informativo mais amplo.

       Identificar e extrair informações chave, dados, tendências e exemplos **que expliquem e detalhem o tópico**.
       Summarizar os achados de forma clara e concisa, priorizando a apresentação das **informações informativas e factuais** encontradas. Mencione recomendações apenas se forem altamente relevantes para o conteúdo informativo.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    resposta_do_agente_buscador = call_agent(buscador, entrada_do_agente_buscador)
    return resposta_do_agente_buscador

In [10]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Função: Agente especializado em estruturar e organizar informações para
        criar planos de conteúdo detalhados e acionáveis para a redação final.
        Baseado no Tópico e nos dados de Lançamentos buscados, crie um plano
        de conteúdo organizado em SEÇÕES ENUMERADAS em Markdown.

        Exemplo de formato desejado:
        1. Título da Primeira Seção (ex: tema 1)
        2. Título da Segunda Seção (ex: tema 2)
        3. Título da Terceira Seção (ex: tema 3)
        4. Titulo da Quarta Seção (ex: tema 4)

        Certifique-se de que cada ponto numerado é um título de seção claro e conciso.
        Filtrar os dados recebidos pelo agente_buscador selecionando apenas 4 mais relevantes e importantes
        abordar.
        NÃO GERAR MAIS DE 4 SEÇÕES.
        Selecionar sitações, pesquisas ou fatos interessantes para abordar nos paragrafos.
        O plano deve ser uma lista numerada que delineia a estrutura completa do post.
        """,
        description="Planejador de conteúdo",
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [11]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático.
        Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
        escreva o texto abordando o tema.
        Importante ressaltar para deixar pelo menos 1 citação relevante.
        O post deve ser engajador, informativo, que prenda a tenção e foco do leitor.
        O texto deve seguir um padrão de no máximo 300 palavras.
            """,
        description="Agente redator de posts engajador"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [12]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um revisor Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático. Seja conciso em suas redações.
        Revise o rascunho de post abaixo sobre o tópico indicado, verificando
        sua escrita.
        Não desejo que seja exigente e nem faça muitas correções
        Apenas exija mudança se for realmente importante e necessário para
        melhoria do material original.
        Se o rascunho estiver bom, responda apenas 'O rascunho está pronto!'.
        O texto deve seguir um padrão de no máximo 300 palavras.
        proibido de fazer correção após 2 tentativas.
        """,
        description="Agente revisor de post."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [13]:
from datetime import date
import re
from IPython.display import display, Markdown
from google.adk.agents import Agent  # Import Agent
from google.adk.runners import Runner # Import Runner
from google.adk.sessions import InMemorySessionService # Import InMemorySessionService
from google.adk.tools import google_search # Import google_search
from google.genai import types  # Para criar conteúdos (Content e Part) # Import types
import textwrap

# --- Funções Auxiliares (mantenha ou adapte conforme necessário) ---
# ... (mantenha as definições de extract_content_from_part, call_agent, to_markdown, extrair_secoes_do_plano)

def extract_content_from_part(part):
    """Extracts text content from a GenAI Part."""
    if part.text:
        return part.text
    return ""

def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    try:
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            if event.is_final_response():
              for part in event.content.parts:
                final_response += extract_content_from_part(part)
                final_response += "\n"
    except Exception as e:
        print(f"Error during agent run: {e}")
    return final_response

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


def extrair_secoes_do_plano(texto_plano: str) -> list[str]:
    secoes = []
    padroes_secao = [
        r'^\s*##\s*(.+)',
        r'^\s*\d+\.\s*(.+)',
        r'^\s*-\s*(.+)'
    ]
    linhas = texto_plano.splitlines()
    for linha in linhas:
        for padrao in padroes_secao:
            match = re.match(padrao, linha.strip())
            if match:
                secao_titulo = match.group(1).strip()
                if secao_titulo and len(secao_titulo) > 3:
                    secoes.append(secao_titulo)
                break
    if not secoes and texto_plano.strip():
         return ["Conteúdo Principal"]
    elif not secoes and not texto_plano.strip():
        return []
    return secoes


# --- Agente 1: Buscador de Notícias ---
# Defina o objeto do agente aqui ou antes da lógica principal
buscador_agent_obj = Agent(
    name="agente_buscador",
    model="gemini-2.0-flash",
    description="Agente que busca informação geral e detalhada no google sobre um tópico.",
    tools=[google_search],
    instruction="""
   Função: Agente especializado em buscar e coletar dados e informações
   relevantes em (Google Search).
   Fornecer pesquisa detalhadas e relevantes para o planejamento e organização de conteúdo.
   Oferecer apenas as pesquisas mais relevantes.
   **Instruções de Busca Específicas:**
   1. **Foco Principal:** Busque por **informações aprofundadas, dados, fatos, tendências, contextos e detalhes** sobre o tópico fornecido, **considerando o foco e propósito especificados pelo usuário**. O objetivo é obter um entendimento completo e factual do assunto, **alinhado com a forma de abordagem desejada**.
   2. **Conteúdo a Excluir Ativamente:**
      - Cursos online, treinamentos, webinars, plataformas de ensino à distância, materiais de curso.
   3. **Recomendações (Livros, etc.):** Se encontrar recomendações relevantes (como livros) que **complementam ou ilustram a informação geral** sobre o tópico, pode incluí-las. No entanto, o foco **não** é criar uma lista de recomendações, mas sim usar essas recomendações como exemplos ou fontes dentro de um contexto informativo mais amplo.

   Identificar e extrair informações chave, dados, tendências e exemplos **que expliquem e detalhem o tópico**, **seguindo a forma de abordagem e o propósito definidos pelo usuário**.
   Summarizar os achados de forma clara e concisa, priorizando a apresentação das **informações informativas e factuais** encontradas. Mencione recomendações apenas se forem altamente relevantes para o conteúdo informativo.
    """
)

# --- Agente 2: Planejador de posts ---
# Defina o objeto do agente aqui ou antes da lógica principal
planejador_agent_obj = Agent(
    name="agente_planejador",
    model="gemini-2.0-flash",
    instruction="""
    Função: Agente especializado em estruturar e organizar informações para
    criar planos de conteúdo detalhados e acionáveis para a redação final.
    Baseado no Tópico, Forma de Abordagem, Propósito e nos dados de Lançamentos buscados, crie um plano
    de conteúdo organizado em SEÇÕES ENUMERADAS em Markdown.

    Exemplo de formato desejado:
    1. Título da Primeira Seção (ex: tema 1)
    2. Título da Segunda Seção (ex: tema 2)
    3. Título da Terceira Seção (ex: tema 3)
    4. Titulo da Quarta Seção (ex: tema 4)

    Certifique-se de que cada ponto numerado é um título de seção claro e conciso.
    Filtrar os dados recebidos pelo agente_buscador selecionando apenas 4 mais relevantes e importantes
    abordar, **considerando a Forma de Abordagem e o Propósito especificados**.
    NÃO GERAR MAIS DE 4 SEÇÕES.
    Selecionar sitações, pesquisas ou fatos interessantes para abordar nos paragrafos, **alinhados com a Forma de Abordagem e Propósito**.
    O plano deve ser uma lista numerada que delineia a estrutura completa do post.
    """,
    description="Planejador de conteúdo",
)

# --- Agente 3: Redator do Post ---
# Defina o objeto do agente aqui ou antes da lógica principal
redator_agent_obj = Agent(
    name="agente_redator",
    model="gemini-2.0-flash",
    instruction="""
        Você é um redator Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático.
        Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
        escreva o texto abordando o tema, **mantendo a Forma de Abordagem e o Propósito originais**.
        Importante ressaltar para deixar pelo menos 1 citação relevante.
        O post deve ser engajador, informativo, que prenda a tenção e foco do leitor.
        O texto deve seguir um padrão de no máximo 300 palavras.

        """,
    description="Agente redator de posts engajador"
)

# --- Agente 4: Revisor de Qualidade ---
# Defina o objeto do agente aqui ou antes da lógica principal
revisor_agent_obj = Agent(
    name="agente_revisor",
    model="gemini-2.0-flash",
    instruction="""
        Você é um revisor Criativo especializado em linguagem e gramatica,
        com foco em criar redações e resumos tecnicos profissionais.
        Você escreve para alunos e profissionais buscando aprimorar seus conhecimentos.
        Escrita profissional mas com toque didático. Seja conciso em suas redações.
        Revise o rascunho de post abaixo sobre o tópico indicado, verificando
        sua escrita, **assegurando que a Forma de Abordagem e o Propósito originais sejam mantidos**.
        Não desejo que seja exigente e nem faça muitas correções
        Apenas exija mudança se for realmente importante e necessário para
        melhoria do material original.
        Se o rascunho estiver bom, responda apenas 'O rascunho está pronto!'.
        O texto deve seguir um padrão de no máximo 300 palavras.
        proibido de fazer correção após 2 tentativas.
        """,
    description="Agente revisor de post."
)


# --- Lógica Principal de Orquestração do Chatbot (Modificada) ---

## 🚀 Iniciando o Sistema de Planejamento e Geração de Conteúdo 🚀

# Obtém a data atual para uso posterior
data_de_hoje = date.today().strftime("%d/%m/%Y")

### ❓ Obtendo o Tópico e Preferências do Usuário

print("## 📚 Inicie o Planejamento do seu Conteúdo! 📚")

print("\nPor favor, digite o TÓPICO principal sobre o qual você quer estudar/criar conteúdo:")
topico = input("Tópico: ")

if not topico:
  print("❌ Nenhum tópico inserido. Por favor, insira um tópico válido para continuar.")
else:
  print(f"\n✅ Tópico recebido: {topico}.")

  print("\nCom qual FORMA DE ABORDAGEM você deseja que o tema seja tratado? (Ex: Técnica, Didática, Histórica, Comparativa, etc.)")
  forma_abordagem = input("Forma de Abordagem: ")

  print("\nQual o PROPÓSITO principal deste conteúdo? (Ex: Explicar um conceito, Comparar tecnologias, Apresentar tendências, Guia prático, etc.)")
  proposito_conteudo = input("Propósito do Conteúdo: ")

  print(f"\n✅ Preferências capturadas:")
  print(f"- Tópico: {topico}")
  print(f"- Forma de Abordagem: {forma_abordagem if forma_abordagem else 'Não especificada'}")
  print(f"- Propósito: {proposito_conteudo if proposito_conteudo else 'Não especificado'}")

  print("\nIniciando o processo de busca e planejamento...")

  ## 🔍 Etapa 1: Buscando Informações Relevantes (Agente Buscador)

  print("\n--- Executando Agente Buscador ---")
  # Chama o agente buscador para coletar dados sobre o tópico
  # Inclua as preferências do usuário na entrada do agente buscador
  entrada_do_agente_buscador = f"""
  Tópico: {topico}
  Data de hoje: {data_de_hoje}
  Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
  Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
  """
  lançamentos_buscados = call_agent(buscador_agent_obj, entrada_do_agente_buscador)

  print("\n### Resultado da Busca:")
  # Exibe os resultados da busca formatados em Markdown
  display(to_markdown(lançamentos_buscados))
  print("-------------------------------")

  ## 📝 Etapa 2: Planejando a Estrutura do Conteúdo (Agente Planejador)

  print("\n--- Executando Agente Planejador ---")
  # Chama o agente planejador para criar um plano de conteúdo baseado na busca
  # Inclua as preferências do usuário na entrada do agente planejador
  entrada_do_agente_planejador = f"""
  Tópico: {topico}
  Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
  Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
  Lançamentos buscados: {lançamentos_buscados}
  """
  plano_de_post = call_agent(planejador_agent_obj, entrada_do_agente_planejador)

  print("\n### Plano de Conteúdo Gerado:")
  # Exibe o plano de conteúdo formatado em Markdown
  display(to_markdown(plano_de_post))
  print("-------------------------------")

  ## ✍️ Etapa 3: Gerando e Revisando Conteúdo por Seção (Agente Redator & Revisor)

  print("\n--- Iniciando Geração e Revisão do Rascunho (por Seção) ---")

  # Extrai as seções do plano de conteúdo para processamento individual
  secoes_do_plano = extrair_secoes_do_plano(plano_de_post)

  # Crie uma lista para armazenar os rascunhos de cada seção
  rascunhos_individuais_secoes = []

  if not secoes_do_plano:
      print("⚠️ Aviso: Não foi possível extrair seções do plano. Não será possível gerar o rascunho por seção.")
      rascunho_do_post = "" # Define rascunho_do_post como vazio se não houver seções
  else:
      rascunho_do_post_partes = [] # Lista para armazenar as partes do rascunho por seção
      MAX_TENTATIVAS_REVISAO = 3 # Limite para o ciclo de revisão por seção

      # Loop sobre cada seção do plano
      for i, secao in enumerate(secoes_do_plano):
          print(f"\n--- Processando Seção {i+1}/{len(secoes_do_plano)}: '{secao}' ---")

          ### ✏️ Gerando Rascunho Inicial da Seção

          # Inclua as preferências do usuário na entrada para o redator
          entrada_para_redator_secao = f"""
          Tópico Geral: {topico}
          Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
          Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}

          Plano de Conteúdo Completo:
          {plano_de_post}

          ---
          POR FAVOR, ESCREVA APENAS O CONTEÚDO DA SEÇÃO ESPECÍFICA ABAIXO.
          Não inclua o título da seção no início da sua resposta.
          Seção a escrever: {secao}

          Instruções Adicionais para esta Seção:
          (Se houver instruções específicas para esta seção do plano, adicione aqui)
          """
          rascunho_da_secao = call_agent(redator_agent_obj, entrada_para_redator_secao)
          print("\n#### Rascunho Inicial:")
          display(to_markdown(rascunho_da_secao))
          print("-------------------------------")

          ### 👀 Ciclo de Revisão da Seção

          revisao_finalizada = False
          tentativas_revisao = 0

          # Loop de revisão até aprovação ou limite de tentativas
          while not revisao_finalizada and tentativas_revisao < MAX_TENTATIVAS_REVISAO:
              tentativas_revisao += 1
              print(f"\n--- Revisando Seção (Tentativa {tentativas_revisao}/{MAX_TENTATIVAS_REVISAO}) ---")

              # Inclua as preferências do usuário na entrada para o revisor
              entrada_para_revisor_secao = f"""
              Tópico Geral: {topico}
              Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
              Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
              Plano de Conteúdo Relevante para esta Seção: {plano_de_post}

              ---
              Rascunho da Seção "{secao}" para Revisão:
              {rascunho_da_secao}

              Por favor, revise o rascunho da seção acima. Se estiver bom, responda APENAS a frase exata 'O rascunho está ótimo e pronto!'. Caso contrário, aponte os problemas e sugira melhorias específicas para que o redator possa corrigir. Certifique-se de que a revisão leva em conta a Forma de Abordagem e o Propósito originais.
              """

              feedback_revisor = call_agent(revisor_agent_obj, entrada_para_revisor_secao)
              print("\n#### Feedback do Revisor:")
              display(to_markdown(feedback_revisor))
              print("-------------------------------")

              if feedback_revisor.strip().lower() == 'o rascunho está ótimo e pronto!':
                  print(f"👍 Seção '{secao}' aprovada pelo Revisor!")
                  revisao_finalizada = True # Sai do loop de revisão para esta seção
              else:
                  print(f"🔄 Seção '{secao}' precisa de correções. Enviando feedback para o Redator.")
                  # Prepara a entrada para o Redator corrigir com base no feedback
                  # Inclua as preferências do usuário na entrada para o redator corrigir
                  entrada_para_redator_correcao = f"""
                  Tópico Geral: {topico}
                  Forma de Abordagem Desejada: {forma_abordagem if forma_abordagem else 'Não especificada'}
                  Propósito do Conteúdo: {proposito_conteudo if proposito_conteudo else 'Não especificado'}
                  Plano de Conteúdo Completo:
                  {plano_de_post}
                  ---
                  POR FAVOR, REESCREVA/CORRIJA APENAS O CONTEÚDO DA SEÇÃO ESPECÍFICA ABAIXO, BASEADO NO FEEDBACK.
                  Não inclua o título da seção no início da sua resposta.
                  Seção a corrigir: {secao}
                  Rascunho Anterior Desta Seção:
                  {rascunho_da_secao}
                  ---
                  Feedback do Revisor para Correção:
                  {feedback_revisor}
                  Por favor, incorpore as sugestões de correção do revisor no novo rascunho desta seção, garantindo que a Forma de Abordagem e o Propósito sejam mantidos.
                  """
                  # Chama o Redator novamente com a solicitação de correção
                  rascunho_da_secao = call_agent(redator_agent_obj, entrada_para_redator_correcao)
                  print("\n#### Rascunho Corrigido:")
                  display(to_markdown(rascunho_da_secao))
                  print("-------------------------------")

          # Fim do ciclo de revisão para a seção atual

          if not revisao_finalizada:
              print(f"⚠️ Aviso: A Seção '{secao}' não foi aprovada após {MAX_TENTATIVAS_REVISAO} tentativas. Usando a última versão gerada.")

          # Adiciona a versão final da seção (aprovada ou a última tentativa) à lista de rascunhos individuais
          # Adiciona também o título da seção para melhor visualização no final
          rascunhos_individuais_secoes.append(f"## {secao}\n\n{rascunho_da_secao}")

          # Adiciona também às partes do rascunho completo (mantendo a funcionalidade original)
          rascunho_do_post_partes.append(f"## {secao}\n\n{rascunho_da_secao}")


      # Junta todas as partes das seções revisadas para formar o rascunho completo
      rascunho_do_post = "\n\n".join(rascunho_do_post_partes)

      print("\n--- Geração e Revisão por Seção Concluídas ---")
      print("\n## 🎉 Rascunho Completo Gerado (Após Revisão por Seção) 🎉")
      # Exibe o rascunho completo final formatado
      display(to_markdown(rascunho_do_post))
      print("-------------------------------")


  ## ✅ Processo de Planejamento e Geração Finalizado ✅

  print("\nO sistema concluiu o processo de planejamento e geração de conteúdo.")

## 📚 Inicie o Planejamento do seu Conteúdo! 📚

Por favor, digite o TÓPICO principal sobre o qual você quer estudar/criar conteúdo:
Tópico: Inteligencia artificial

✅ Tópico recebido: Inteligencia artificial.

Com qual FORMA DE ABORDAGEM você deseja que o tema seja tratado? (Ex: Técnica, Didática, Histórica, Comparativa, etc.)
Forma de Abordagem: Dídatica e técnica

Qual o PROPÓSITO principal deste conteúdo? (Ex: Explicar um conceito, Comparar tecnologias, Apresentar tendências, Guia prático, etc.)
Propósito do Conteúdo: Apresentar tendência

✅ Preferências capturadas:
- Tópico: Inteligencia artificial
- Forma de Abordagem: Dídatica e técnica
- Propósito: Apresentar tendência

Iniciando o processo de busca e planejamento...

--- Executando Agente Buscador ---

### Resultado da Busca:


> Ok, compreendido. Vou realizar uma pesquisa detalhada sobre inteligência artificial, com foco em apresentar as tendências atuais de forma didática e técnica.
> 
> 
> Compreendido. Com base nas minhas buscas, aqui estão as principais tendências e informações relevantes sobre Inteligência Artificial (IA) em 2025, apresentadas de forma didática e técnica, com foco nas tendências e no propósito de apresentar este conteúdo:
> 
> **1. Tendências Transformadoras da IA em 2025:**
> 
> *   **Democratização da IA:** A IA está se tornando mais acessível a empresas de todos os tamanhos, impulsionada por ferramentas *no-code* e *low-code* que eliminam a necessidade de conhecimento técnico avançado. Isso permite que mais empresas criem soluções personalizadas, como automação de fluxos, análises preditivas e dashboards interativos. Estima-se que, até 2026, mais de 80% dos fornecedores de software integrarão IA generativa em seus produtos, e 70% das novas aplicações serão desenvolvidas com ferramentas *no-code* ou *low-code*.
> *   **IA Generativa em Expansão:** A IA generativa, capaz de criar textos, imagens e código de forma autônoma, continuará a se expandir e transformar negócios em áreas como marketing, design e desenvolvimento de produtos.
> *   **Automação Inteligente:** A automação inteligente, impulsionada pela IA, revoluciona processos ao simplificar, automatizar e otimizar operações de maneira estratégica. Ao combinar aprendizado de máquina, análise avançada e algoritmos, as empresas eliminam tarefas manuais repetitivas, reduzem custos operacionais e elevam a eficiência.
> *   **Agentes de IA:** Agentes de IA estão se tornando "membros" das equipes, realizando tarefas operacionais e liberando humanos para atividades estratégicas. Esses agentes podem tomar decisões e realizar ações de forma autônoma, aliviando a sobrecarga de tarefas.
> *   **Hiperpersonalização:** A IA possibilita a criação de experiências hiperpersonalizadas para clientes e fornecedores, aumentando a transparência e a colaboração. Isso é alcançado por meio de análises avançadas de dados, *machine learning* e automação.
> *   **IA Multimodal:** A IA multimodal consegue processar diferentes tipos de dados ao mesmo tempo, como texto, imagem, áudio e vídeo, permitindo uma leitura mais completa do contexto.
> *   **IA Colaborativa:** A IA está se tornando mais colaborativa, com agentes de IA atuando como assistentes de profissionais, permitindo que eles gerenciem melhor seu tempo e criem espaço para o pensamento estratégico e a criatividade.
> *   **IA em Saúde e Biotecnologia:** A IA na saúde está avançando em diagnósticos assistidos por IA, análise de imagens médicas e desenvolvimento de medicamentos.
> *   **Sustentabilidade Inteligente:** A IA está sendo usada para criar soluções de sustentabilidade inteligente, ajudando as empresas a reduzir seu impacto ambiental.
> *   **IA na Segurança:** A IA está sendo utilizada no setor de segurança para melhorar a detecção de ameaças e proteger dados.
> *   **IA Conversacional Sofisticada:** A IA conversacional está se tornando mais sofisticada, com chatbots e assistentes virtuais capazes de entender e responder a perguntas de forma mais natural e eficaz.
> *    **Computação Quântica:** A integração da IA com a computação quântica pode ampliar exponencialmente a capacidade de processamento e resolver problemas complexos com maior eficiência.
> *   **IA em IoT e Edge Computing:** A crescente presença da IA em dispositivos IoT (Internet das Coisas) e no *edge computing* permitirá que dispositivos inteligentes processem dados localmente, reduzindo a latência e melhorando a eficiência.
> 
> **2. Impacto da IA no Mercado de Trabalho:**
> 
> *   A IA está remodelando setores inteiros e criando oportunidades que antes eram inimagináveis.
> *   Espera-se que a IA crie cerca de 97 milhões de novos empregos até 2025, ao mesmo tempo em que elimina aproximadamente 85 milhões de postos de trabalho tradicionais.
> *   Profissões ligadas à análise de dados, desenvolvimento de IA, segurança cibernética e experiência do cliente devem crescer significativamente.
> *   A automação centrada no humano está se tornando uma tendência, com ferramentas intuitivas que aprendem com as preferências do usuário e oferecem sugestões personalizadas.
> *   A educação continuará a ser uma prioridade, com ferramentas como tutores virtuais e plataformas de aprendizagem adaptativa utilizando IA para criar percursos de aprendizagem personalizados.
> *   A requalificação profissional é essencial para acompanhar as mudanças do mercado, com foco em habilidades como criatividade, empatia e pensamento crítico.
> *   A IA não substitui o trabalho humano, mas redefine suas funções, impulsionando a reinvenção do trabalho e criando um paradoxo onde, quanto mais a tecnologia avança, mais valorizamos o que é genuinamente humano.
> 
> **3. Ética e Governança da IA:**
> 
> *   A ética será uma preocupação central, com empresas adotando práticas mais transparentes para garantir que os sistemas de IA sejam justos, imparciais e respeitem a privacidade.
> *   A regulamentação da IA está se tornando mais rigorosa, com iniciativas globais para garantir o uso responsável da tecnologia.
> *   Questões éticas, como viés algorítmico e privacidade de dados, continuam em debate, impulsionando a necessidade de auditorias e regulamentações mais rigorosas.
> *   O Regulamento da IA da UE proíbe oito práticas, incluindo manipulação e engano prejudiciais com base na IA, exploração nociva de vulnerabilidades com base na IA e pontuação social.
> *   A responsabilidade legal em caso de erros ou danos causados por sistemas de IA é uma questão importante, com debates sobre quem é o responsável em casos de acidentes ou decisões judiciais erradas.
> *   A transparência e a contestabilidade são princípios essenciais na regulamentação da IA, com a necessidade de mecanismos claros para contestar decisões influenciadas por IA.
> 
> **4. Aplicações da IA em Diversos Setores:**
> 
> *   **Saúde:** Diagnósticos mais precisos, tratamentos personalizados, análise de imagens médicas e desenvolvimento de medicamentos.
> *   **Finanças:** Detecção de fraudes, gestão de investimentos, análise de risco de crédito e personalização de serviços financeiros.
> *   **Logística:** Otimização de rotas, previsão de demandas, gestão de estoques e automação de armazéns.
> *   **Manufatura:** Automação de processos, controle de qualidade e manutenção preditiva.
> *   **Varejo:** Personalização de experiências do cliente, otimização de preços e gestão de estoque.
> *   **Educação:** Aprendizado adaptativo, tutores virtuais e automação de tarefas administrativas.
> *   **Setor Público:** Previsão de eventos climáticos extremos, análise de dados climáticos e otimização de serviços públicos.
> 
> **5. Desafios e Riscos da IA:**
> 
> *   Viés algorítmico e discriminação.
> *   Privacidade e proteção de dados.
> *   Responsabilidade legal e ética.
> *   Segurança cibernética.
> *   Necessidade de requalificação profissional.
> *   Impacto na segurança nacional.
> 
> **6. Regulamentação da IA no Brasil:**
> 
> *   O projeto de Lei 2.338/2023, conhecido como Marco da Inteligência Artificial, está em discussão na Câmara dos Deputados.
> *   O Conselho Nacional de Justiça (CNJ) aprovou uma nova resolução que estabelece diretrizes para o uso da IA no Poder Judiciário.
> *   A regulamentação da IA no Brasil ainda está em desenvolvimento e busca garantir a proteção de direitos fundamentais, a segurança jurídica e as prerrogativas da advocacia.
> 
> **7. Oportunidades para Startups e Empresas:**
> 
> *   A IA oferece uma série de vantagens para startups e empresas, como a automação de processos, a análise de grandes volumes de dados, a criação de *chatbots* para atendimento ao cliente e a inovação em produtos e serviços.
> *   Startups de IA estão recebendo financiamento crescente, indicando um ambiente propício para o lançamento de novos empreendimentos no espaço de IA.
> *   A escassez de profissionais especializados em IA sinaliza uma oportunidade lucrativa para quem busca aprimorar ou reciclar habilidades em tecnologias de IA.
> *   O setor de IA está em crescimento e oferece diversas oportunidades para empresas de todos os tamanhos e setores.
> 
> Espero que esta informação detalhada e organizada seja útil para o seu planejamento de conteúdo.
> 


-------------------------------

--- Executando Agente Planejador ---

### Plano de Conteúdo Gerado:


> Perfeito! Com base nas informações detalhadas e na forma de abordagem didática e técnica, aqui está um plano de conteúdo estruturado em seções numeradas em Markdown, focado em apresentar as tendências da Inteligência Artificial em 2025:
> 
> 1.  **IA em 2025: Panorama das Tendências Transformadoras**
>     *   Abordar a democratização da IA com ferramentas no-code/low-code.
>     *   Discutir a expansão da IA generativa e seus impactos em diversas áreas.
>     *   Explicar a automação inteligente e como ela otimiza processos empresariais.
>     *   Introduzir o conceito de agentes de IA como membros das equipes de trabalho.
> 2.  **Impacto da IA no Mercado de Trabalho e Desenvolvimento Profissional**
>     *   Analisar como a IA está remodelando setores e criando novas oportunidades.
>     *   Discutir o aumento de empregos em análise de dados, desenvolvimento de IA e segurança cibernética.
>     *   Abordar a importância da requalificação profissional e habilidades como criatividade e empatia.
>     *   Explicar como a IA redefine funções, valorizando o que é genuinamente humano.
> 3.  **Ética e Governança da IA: Desafios e Regulamentação**
>     *   Destacar a importância da ética e da transparência no uso da IA.
>     *   Discutir a regulamentação da IA e iniciativas para garantir o uso responsável da tecnologia.
>     *   Analisar questões como viés algorítmico, privacidade de dados e responsabilidade legal.
>     *   Mencionar o Regulamento da IA da UE e o debate sobre transparência e contestabilidade.
> 4.  **Aplicações da IA em Setores-Chave e Oportunidades para o Futuro**
>     *   Apresentar exemplos de aplicações da IA em saúde, finanças, logística, manufatura, varejo, educação e setor público.
>     *   Discutir como a IA está sendo utilizada para diagnósticos precisos, personalização de serviços e otimização de processos.
>     *   Abordar as oportunidades para startups e empresas no desenvolvimento e aplicação de soluções de IA.
>     *   Destacar a importância de profissionais especializados em IA e o potencial de crescimento no setor.
> 


-------------------------------

--- Iniciando Geração e Revisão do Rascunho (por Seção) ---

--- Processando Seção 1/4: '**IA em 2025: Panorama das Tendências Transformadoras**' ---

#### Rascunho Inicial:


...

Exception in thread Thread-64 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u


#### Rascunho Corrigido:


-------------------------------
⚠️ Aviso: A Seção '**Aplicações da IA em Setores-Chave e Oportunidades para o Futuro**' não foi aprovada após 3 tentativas. Usando a última versão gerada.

--- Geração e Revisão por Seção Concluídas ---

## 🎉 Rascunho Completo Gerado (Após Revisão por Seção) 🎉


> ## **IA em 2025: Panorama das Tendências Transformadoras**
> 
> Em 2025, a Inteligência Artificial (IA) se firma como a força motriz da inovação. A democratização da IA ganha tração com ferramentas no-code/low-code, permitindo que usuários de diversas áreas implementem soluções inteligentes sem a necessidade de expertise em programação.
> 
> A IA generativa expande-se, revolucionando a criação de conteúdo e a descoberta de novos materiais, com aplicações que vão da arte à pesquisa científica. A automação inteligente redefine a eficiência empresarial, otimizando processos e permitindo que profissionais se concentrem em atividades estratégicas e criativas.
> 
> Agentes de IA surgem como membros de equipes, auxiliando na tomada de decisões e elevando a produtividade. Andrew Ng, referência em IA, resume: "IA é a nova eletricidade". Assim como a eletricidade transformou indústrias, a IA molda o presente e o futuro.
> 
> 
> 
> ## **Impacto da IA no Mercado de Trabalho e Desenvolvimento Profissional**
> 
> A Inteligência Artificial (IA) está remodelando o mercado de trabalho, criando novas oportunidades e exigindo adaptação profissional. Setores como análise de dados, desenvolvimento de IA e segurança cibernética estão em expansão, impulsionados pela crescente busca por soluções inteligentes e inovadoras.
> 
> A requalificação profissional torna-se essencial, com ênfase em habilidades como criatividade, pensamento crítico e empatia, que complementam as capacidades da IA. A IA redefine funções, automatizando tarefas repetitivas e valorizando o que é inerentemente humano.
> 
> Conforme destaca Andrew Ng, renomado especialista em IA, "A IA é a nova eletricidade. Assim como a eletricidade transformou diversas indústrias há um século, a IA agora tem o potencial de fazer o mesmo".
> 
> 
> 
> ## **Ética e Governança da IA: Desafios e Regulamentação**
> 
> 
> 
> ## **Aplicações da IA em Setores-Chave e Oportunidades para o Futuro**
> 


-------------------------------

O sistema concluiu o processo de planejamento e geração de conteúdo.
